<a href="https://colab.research.google.com/github/sergiokv13/03MAIR-Algoritmos-de-Optimizacion/blob/master/tree/master/SEMINARIOSeminario_Sergio_K%C3%B6ller_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
<strong>Nombre y Apellidos:</strong> Sergio Andres Köller Vargas <br>
<strong>Url:</strong> https://colab.research.google.com/drive/1NC7__Axzw9pypgs8-xQzfKhnBgATRN49?usp=sharing<br>
<strong>Problema:</strong>
> 1. Sesiones de doblaje <br>

<strong>Descripción del problema:</strong>

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio
de grabación independientemente del número de tomas que se graben. No es posible grabar
más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto
por los servicios de los actores de doblaje sea el menor posible.<br><br>
Numero de actores: 10<br>
Numero de tomas: 30<br>
Actores/Tomas: [Documento](https://bit.ly/36D8IuK)<br>
<em>* 1 indica que el actor participa en la toma 0 el caso contrario</em><br><br>



                                        

In [15]:
import random
import copy
import math
import itertools 
from functools import reduce
import operator as op

## Conteo de Posibilidades
### ¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>

La restricción con la que nos encontramos es que <strong>no es posible realizar mas de 6 tomas al dia.</strong> Las demas "restricciones" son peculiaridades del problema, que sin ellas este no tendria un sentido. Es decir, nuestro problema sin restricciones es:

Los actores deben coincidir en las tomas en las que sus personajes aparecen juntos (sin esto no seria posible realizar una toma).
Los actores cobran lo mismo por día que deban desplazarse independientemente del número de tomas.

Nuestras soluciones seran la manera de distribuir nuestras tomas en diferentes días.

Tomaremos solo los diás en los que se realiza al menos una toma para realizar nuestro conteo. 

Por día podriamos realizar desde 1 a 30 tomas (no hay restriccion de 6 tomas por dia y 0 tomas no cuenta como un día valido). No nos importa el orden de las tomas en el día, solo nos importa como dividimos esas tomas entre 1 a 30 días.

Ejemplificación:
- 30 tomas en dia 1.
- 29 tomas en dia 1 y 1 toma en dia 2.
- 28 tomas en dia 1 y 2 tomas en dia 2.
- 28 tomas en dia 1, 1 toma en dia 2, 1 toma en dia 3
...

Es decir nuestro problema sera el como escoger x tomas de las que no asignamos aun multiplicado por el como distribuir las x-n tomas restantes. Siendo n las opciones de distribuir las tomas restantes en diferentes días.

Esta representación puede ser definida recursivamente.

```
f(n) = 1 donde n <= 1
f(n) = C(n,1) * f(n-1) + C(n,2) * f(n-2) + ... + C(n,n-1) * f(1) + c(n,n) * f(0)
```

f(0) es igual a 1 para poder contar la opción en la que tomamos todas las tomas en un mismo día.

De manera mas elegante:

$$ f(n)=  \left\{
\begin{array}{ll}
      1 & n <= 1 \\
      \sum_{x=1}^{n} C\binom{n}{x}*f(n-x) & n > 1 \\
\end{array} 
\right. $$ 

donde n es 30.

In [16]:
# Calculamos las posibilidades programaticamente
def nCr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

# Haremos uso de programación dinamica para no recalcular
posibilities_memo = {}
def get_posibilities(n = 30):
  if n <= 1: return 1
  if n in posibilities_memo: return posibilities_memo[n]

  posibilities = 0
  for x in range(1, n+1):
    posibilities += nCr(n,x) * get_posibilities(n-x)

  posibilities_memo[n] = posibilities
  return posibilities

print("El número de posibilidades es: ", get_posibilities())


El número de posibilidades es:  11403568794011880483742464196184901963


### ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?

La principal restricción con la que nos encontramos es que <strong>no es posible realizar mas de 6 tomas al día.</strong>. Es decir el objetivo es el de 6 o menos tomas a realizarse en un dia especifico. 

Agregando la restricción de solo poder <strong>realizar 6 tomas al dia</strong> nuestro espacio de posibilidades se ve reducido. Ahora tenemos la opción de tomar entre 1 a 6 tomas por dia.

Al igual que en el anterior punto, definiremos recursivamente.

```
f(n) = 1 si n = 1
```
La idea es elegir x tomas en este día multiplicado por todas las maneras en que distribuiremos n-x. Sumamos todas las opciones tomando x desde 1 hasta 6 (maximo número de tomas al día). Para n's menores a 6, solo podremos distribuir desde x hasta n.
```
f(n) = f(n-1) * C(n,1) + f(n-2) * C(n,2) si n = 2
f(n) = f(n-1) * C(n,1) + f(n-2) * C(n,2) + f(n-3) * C(n,3) + f(n-4) si n = 3
f(n) = f(n-1) * C(n,1) + f(n-2) * C(n,2) + f(n-3) * C(n,3) + f(n-4) * C(n,4)   si n = 4
f(n) = f(n-1) * C(n,1) + f(n-2) * C(n,2) + f(n-3) * C(n,3) + f(n-4) * C(n,4) + f(n-5) * C(n,5)   si n = 5
```
Para todos los n's mayores, a 6, entonces podemos distribuir de 1 a 6 tomas por día.
```
f(n) = f(n-1) * C(n,1) + f(n-2) * C(n,2) + f(n-3) * C(n,3) + f(n-4) * C(n,4) + f(n-5) * C(n,5) + f(n-6) * C(n,6) n>= 6
```

Al igual que en el anterior punto, podemos f(0) como 1 para tomar en cuenta la opción de tomar todas las opciones en un día.

De manera mas elegante:

$$ f(n)=  \left\{
\begin{array}{ll}
      1 & n<=1 \\
      \sum_{x=1}^{min(n,6)} C\binom{n}{x}*f(n-x) & n >1 \\
\end{array} 
\right. $$ 

donde n es 30.



In [17]:
# Calculamos las posibilidades programaticamente

# Haremos uso de programación dinamica para no recalcular
posibilities_memo_max_6 = {}
def get_posibilities_max_6(n=30):
  if n <= 1: return 1
  if n in posibilities_memo_max_6: return posibilities_memo_max_6[n]

  posibilities = 0
  for x in range(1, min(n, 6) + 1):
    posibilities += get_posibilities_max_6(n-x) * nCr(n, x)

  posibilities_memo_max_6[n] = posibilities
  return posibilities

print("El número de posibilidades es: ", get_posibilities_max_6(30))

El número de posibilidades es:  11400192964820635839099878586991366320


## Modelo para el espacio de soluciones<br>
### ¿Cual es la estructura de datos que mejor se adapta al problema?


El problema utilizara las siguientes estructuras para almacenar la información necesaria.

<strong>Actores que forman parte de la toma:</strong><br>
Se utilizara un array de numeros. El id del array correspondera al id de la toma. Cada numero almacenado en el array representara una cadena binaria en la que el valor del bit x correspondera a la pertinencia del actor x en la toma.

Ejemplo para 6 tomas y 3 actores:
```
     Actores 0  1  2
0  4     ->  1  0  0
1  4     ->  1  0  0
2  2     ->  0  1  0
3  1     ->  0  0  1
4  4     ->  1  0  0
5  3     ->  0  1  1
```
El uso de un array es eficiente en terminos de memoria vs una matriz, y nos permitira tener un acceso instantaneo a nuestros valores. Por otro lado se decidio hacer uso de manipulación de bits para poder usar operadores sobre bits al momento de calcular nuestra función de coste. Esto nos libra de tener que estar interando sobre cada actor por toma para ver el coste total de un día. <br>

<strong>Solución:</strong><br>
Para la solución haremos uso de un array de sets. Cada posición del array representara un dia (pese a que este orden no afecte al coste total), mientras que cada valor del array sera un set de tomas (tampoco importa el orden de las tomas).
```
{{0,1,2},{3,4,5}}
{{0},{1},{2,3},{4}}
{{0,1,2,3,4}}
```
El uso de sets nos permite no preocuparnos por el orden de las tomas. Tambien nos permitira hacer uso de funciones de sets sin necesidad de estar haciendo cast, cosa que puede ser muy util para la implementación de nuestro algoritmo genetico. Por ultimo nos asegura el control automatico de no repetir una toma dentro de un mismo dia.



In [18]:
# Nuestra matriz de pertenencia
actors_by_take_matrix = [
[1,1,1,1,1,0,0,0,0,0],
[0,0,1,1,1,0,0,0,0,0],
[0,1,0,0,1,0,1,0,0,0],
[1,1,0,0,0,0,1,1,0,0],
[0,1,0,1,0,0,0,1,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,0,0,1,0,0,0,0],
[1,1,0,1,0,0,0,0,0,0],
[1,1,0,0,0,1,0,0,1,0],
[1,1,1,0,1,0,0,1,0,0],
[1,1,1,1,0,1,0,0,0,0],
[1,0,0,1,1,0,0,0,0,0],
[1,0,1,0,0,1,0,0,0,0],
[1,1,0,0,0,0,1,0,0,0],
[0,0,0,1,0,0,0,0,0,1],
[1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0],
[1,0,1,0,0,0,0,0,0,0],
[1,0,1,1,1,0,0,0,0,0],
[0,0,0,0,0,1,0,1,0,0],
[1,1,1,1,0,0,0,0,0,0],
[1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0],
[1,1,0,1,0,0,0,0,0,1],
[1,0,1,0,1,0,0,0,1,0],
[0,0,0,1,1,0,0,0,0,0],
[1,0,0,1,0,0,0,0,0,0],
[1,0,0,0,1,1,0,0,0,0],
[1,0,0,1,0,0,0,0,0,0]]

# función que nos permitira convertir nuestra matriz en un array de numeros
def get_bits_actors(matrix):
  converted_arr = []
  for row in matrix:
    # convertimos el array de bits a un numero
    bin_as_number = int("".join(str(i) for i in row),2)
    converted_arr.append(bin_as_number)
  return converted_arr

# ejemplificamos una solucion para los datos
example_solution = [{1,2,3,4,5},{6,7,8,9,10},{11,12,13,14,15},{16,17,18,19,20},{21,22,23,24,25},{26,27,28,29,0}]

# obtenemos nuestra representación de pertenencia
actors_by_take = get_bits_actors(actors_by_take_matrix)

## Según el modelo para el espacio de soluciones<br>
### ¿Cual es la función objetivo?

Asignaremos un valor constante de 1 al monto que cada actor cobrara por presentarse a un día de doblaje. Entonces haciendo esto, podemos decir que el costo de cada día es la cantidad de actores necesarios en todas las tomas de dicho día. El coste total sera la suma de los costos diarios.

A continuación la implementación de nuestra funcion objetivo.

In [19]:
def cost_function(solution, actors_by_take = actors_by_take):
  total_cost = 0
  for day in solution:
    # almacenamos un número que controle el resultado de ir aplicando el operador | para todos las tomas del día
    # https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Bitwise_OR
    # inicializamos el valor en 0000000000, el cual seria el neutro para nuestros 10 actores
    day_value = 0
    for take in day: 
      day_value |= actors_by_take[take]
    # obtenemos el número de 1's en nuestro valor final, lo cual corresponderia al coste del dia
    day_cost = bin(day_value).count("1")
    total_cost += day_cost
  
  return total_cost

print("Prueba sobre el ejemplo: ", cost_function(example_solution))
        

Prueba sobre el ejemplo:  43


### ¿Es un problema de maximización o minimización?

El objetivo es el de minizar la función de coste previamente espeficificada. Entonces es un problema de minimización.

## Fuerza bruta
#### Diseña un algoritmo para resolver el problema por fuerza bruta

Al ver el espacio de soluciones, podemos ver que un algoritmo de fuerza bruta (Visitar todas las soluciones) no es una opción. Es por esto que para probar nuestro algoritmo de fuerza bruta de manera que pueda terminar y arrojar una solución, reduciremos el problema.

Haremos uso de 3 actores en 6 tomas, con un limite de 3 tomas por dia (Ejemplo presentado en la sección de la estructura de datos).

Notar que nuestro algoritmo recibe como parametros los actores y el maximo numero de tomas por día, por lo cual puede ser aplicado tambien para otras situaciones.

```
     Actores 0  1  2
0  4     ->  1  0  0
1  4     ->  1  0  0
2  2     ->  0  1  0
3  1     ->  0  0  1
4  4     ->  1  0  0
5  3     ->  0  1  1
```

In [20]:
def brute_force(
    actors_by_take, # nuestro array de ints que representan los actores necesarios por toma
    max_takes_on_day # Maximo numero de tomas a realizar por día
  ):

  brute_force.solutions_count = 0
  brute_force.best_solution = []
  brute_force.min_cost = float("inf")

  def brute_force_rec(
      solution=[],  # Almacenara la mejor solución encontrada hasta el momento
    ):
    
    # Las escenas ya tomadas es la union de todos nuestros subsets de la solución
    takes_taken = set().union(*solution)

    # Si todas nuestras tomas ya fueron asignadas, entonces estamos en una solución
    if len(takes_taken) == len(actors_by_take):
      brute_force.solutions_count += 1
      local_cost = cost_function(solution, actors_by_take)
      # Si nuestro coste es menor que el minimo, entonces acabamos de encontrar una mejor solución
      if local_cost <= brute_force.min_cost:
        brute_force.min_cost = local_cost
        brute_force.best_solution = solution

    # Nuestras tomas actuales seran todas menos las que ya fueron tomadas
    takes = set(range(len(actors_by_take))) - takes_taken

    # Podemos armar grupos de 1,2,..,maximo de tomas por dia
    for i in range(1, max_takes_on_day+1):
      # genereramos todos los subconjuntos de tamaño i
      sub_sets = set(itertools.combinations(takes, i))
      for sub_set in sub_sets:
        # llamamos recursivamente a brute_force con nuestra solucion parcial
        brute_force_rec(solution + [sub_set])

  # Llamamos a nuestra funcion recursiva y retornamos
  brute_force_rec()
  print("La mejor distribución es: ", brute_force.best_solution)
  print("Con costo: ", brute_force.min_cost)
  print(f"Se evaluaron {brute_force.solutions_count} soluciones")

In [21]:
# Usamos un caso de ejemplo para ver si el algoritmo es efectivo en revisar todas las soluciones
# usaremos 6 tomas y un máximo de 6 tomas al día

verify_solutions_actors_by_take = [4,4,2,1,4,3]
brute_force(verify_solutions_actors_by_take, 6)
print("El número de posibilidades calculado es: ", get_posibilities_max_6(6))

La mejor distribución es:  [(0, 1, 2, 3, 4, 5)]
Con costo:  3
Se evaluaron 4683 soluciones
El número de posibilidades calculado es:  4683


In [22]:
# Probamos nuestra función de fuerza bruta con 6 tomas y 3 actores, y un maximo de 2 tomas por dia.

small_actors_by_take = [4,4,2,1,4,3]

brute_force(small_actors_by_take, 2)

La mejor distribución es:  [(3, 5), (0, 2), (1, 4)]
Con costo:  5
Se evaluaron 3690 soluciones


### Calcula la complejidad del algoritmo

Haremos una estimación para hacer el calculo de la complejidad de este algoritmo. Podemos ver que visitaremos el espacio de soluciones en el que tomamos una toma por día a lo largo de n días. Siendo n el número de tomas. Este espacio de soluciones tiene complejidad de n!. Si consideramos nuestro maximo numero de tomas por día como una constante multiplicativa, entonces la complejidad es:


```
O(n) = n!
```

Por otro lado si consideramos nuestro máximo número de tomas en nuestra complejidad, entonces tenemos que contar las permutaciones de las formas de armar estos grupos. Siendo m el máximo número de tomas, entonces podremos armar n/m grupos. La complejidad tomando en cuenta m seria:

```
O(n,m) = n!(n/m)!
```




## Mejora del algoritmo
### Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta.

Haremos uso de un algoritmo genetico. Este podra mejorar el algoritmo de fuerza bruta sin ningún problema, ya que al hacer uso de una heuristica, no sera necesario navegar todo el espacio de soluciones. Haciendo uso de un algoritmo genetico podremos conocer una buena solución (No sabremos si es la mejor) de manera eficiente en tiempo y en memoria, cosa que sería imposible con el algoritmo de fuerza bruta para una dimensionalidad tan grande como lo es 30 tomas. (Ver espacio de soluciones en el primer punto)

#### Mutación

El cambio producido por la mutación necesita ser pequeño para que esta nos permita la intensificación. Podemos asumir que tomar mas tomas al dia siempre sera mejor que tomarlas por separado. Esto es ya que el tomar mas tomas tendremos entre 0 o mas intersecciones entre los actores necesarios, mientras que tomarlas por separado no tendra la reducción de interseccionalidad. 

Si nuestro número de tomas es divisible entre el máximo número de tomas, entonces podemos asumir que la solución se encontrara en n grupos, siendo n tomas/maximo_número de tomas. Cosa que no podemos asumir si esta división no es exacta. 

Queremos que nuestra mutación sea aplicable para cualquiera de estos casos, razón por la que al mutar combinaremos días siempre que esten por debajo del máximo de tomas diarias. Para esto haremos una ordenación de los dias por tamaño de tomas, y agruparemos de manera voraz. Esto permitira que nuestra solución converja siempre en soluciones con elementos agrupados.

```
Ejemplo mutación 6 tomas y máximo de 6
[{1,2},{3,4},{5,6}] => [{1,2,3,4,5,6}]
Ejemplo mutación 6 tomas y máximo de 5
[{1,2},{3,4},{5,6}] => [{1,2,3,4},{5,6}]
Ejemplo mutación 6 tomas y máximo de 5 (diferentes tamaños)
[{1},{2,3},{4,5,6}] => [{1,2,3},{4,5,6}]
```
Despues de agrupar, elegiremos dos dias al azar, los uniremos, y los separaremos al azar en partes iguales (si es posible).


In [23]:
def mutation(solution, max_takes_on_day):
  new_solution = []
  # Agrupamos

  solution.sort(key=lambda x:len(x))
  current_set = set()
  for day in solution:
    if len(current_set | day) <= max_takes_on_day:
      # Agregamos el nuevo set si no excede el tamaño
      current_set = current_set | day
    else:
      if len(current_set) > 0:
        # Si agregar el set excede el tamaño, entonces lo consideramos un nuevo dia, y continuamos
        new_solution.append(current_set)
        current_set = day

  # Verificamos si tenemos algo mas que agregar
  if len(current_set) > 0: new_solution.append(current_set)
  # Elegimos dos sets al azar
  elems = random.sample(new_solution, 2)
  indexes = [new_solution.index(elems[0]), new_solution.index(elems[1])]
  # Unimos los elementos y los barajamos
  elems_join = list(set().union(*elems))
  random.shuffle(elems_join)
  # Dividimos por la mitad
  cut_index = len(elems_join) // 2
  cuts = [set(elems_join[:cut_index]), set(elems_join[cut_index:])]
  new_solution[indexes[0]] = cuts[0]
  new_solution[indexes[1]] = cuts[1]

  return new_solution

In [24]:
# Prueba de nuestra mutación
print("prev: ", [{1,2},{3},{4,5,6,7},{8,9,10},{11,12,13,14,15},{16,17},{18,19,20},{21},{22},{23},{24,25},{26,27},{28,29,0}])
print("mutated: ", mutation([{1,2},{3},{4,5,6,7},{8,9,10},{11,12,13,14,15},{16,17},{18,19,20},{21},{22},{23},{24,25},{26,27},{28,29,0}], 6))
print()
print("prev: ", [{1, 2, 23, 28, 29}, {16, 17, 24, 25, 26, 27}, {18, 19, 20, 8, 9, 10}, {0, 3, 21, 22}, {4, 5, 6, 7}, {11, 12, 13, 14, 15}])
print("mutated: ", mutation([{1, 2, 23, 28, 29}, {16, 17, 24, 25, 26, 27}, {18, 19, 20, 8, 9, 10}, {0, 3, 21, 22}, {4, 5, 6, 7}, {11, 12, 13, 14, 15}], 6))

prev:  [{1, 2}, {3}, {4, 5, 6, 7}, {8, 9, 10}, {11, 12, 13, 14, 15}, {16, 17}, {18, 19, 20}, {21}, {22}, {23}, {24, 25}, {26, 27}, {0, 28, 29}]
mutated:  [{1, 2, 3, 21, 22, 23}, {0, 16, 24, 25, 27}, {18, 19, 20, 8, 9, 10}, {17, 26, 28, 29}, {4, 5, 6, 7}, {11, 12, 13, 14, 15}]

prev:  [{1, 2, 23, 28, 29}, {16, 17, 24, 25, 26, 27}, {8, 9, 10, 18, 19, 20}, {0, 3, 21, 22}, {4, 5, 6, 7}, {11, 12, 13, 14, 15}]
mutated:  [{0, 3, 21, 22}, {4, 28, 6, 7}, {1, 2, 5, 23, 29}, {11, 12, 13, 14, 15}, {16, 17, 24, 25, 26, 27}, {8, 9, 10, 18, 19, 20}]


#### Generar soluciones aleatorias

Necesitamos un metodo que genere soluciones de manera aleatoria, esto nos servira para nuestra población inicial. Para esto vamos a barajear las tomas, y separaremos aleatoriamente en [1,maxima tomas por dia] partes iguales.

In [25]:
# Dividir una lista en n partes
def split(a, n):
  k, m = divmod(len(a), n)
  return [set(a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)]) for i in range(n)]
  
# Generar soluciones alteatorias
def generate_random_solutions(quantity, number_of_takes, max_takes_on_day):
  solutions = []
  takes = list(range(number_of_takes))

  for i in range(quantity):
    random.shuffle(takes)
    splits = len(takes) // random.randint(1, max_takes_on_day)
    solutions.append(split(takes, splits))

  return solutions

In [26]:
# prueba de nuestra generacion de soluciones
test_solutions = generate_random_solutions(5, 30, 6)
for sol in test_solutions:
  print(sol)

[{24, 3}, {8, 4}, {27, 5}, {18, 12}, {9, 26}, {10, 6}, {17, 15}, {25, 19}, {28, 13}, {16, 20}, {1, 23}, {2, 7}, {29, 21}, {0, 22}, {11, 14}]
[{3, 4, 8, 20, 27}, {2, 10, 21, 24, 25}, {1, 6, 7, 19, 22}, {5, 12, 14, 17, 18}, {0, 9, 15, 16, 28}, {11, 13, 23, 26, 29}]
[{1, 27, 29}, {26, 5, 14}, {25, 20, 7}, {9, 17, 6}, {16, 0, 23}, {19, 4, 22}, {8, 11, 28}, {3, 13, 15}, {24, 10, 21}, {18, 2, 12}]
[{1, 5, 16, 17, 18, 24}, {0, 2, 7, 8, 13, 22}, {10, 14, 19, 20, 26, 29}, {3, 21, 23, 25, 27, 28}, {4, 6, 9, 11, 12, 15}]
[{3, 12, 15, 20, 26}, {6, 10, 11, 18, 21}, {4, 14, 17, 19, 22}, {16, 25, 27, 28, 29}, {0, 2, 7, 9, 13}, {1, 5, 8, 23, 24}]


#### Selección

Para selecciónar nuestro metodo recibira una población, un número de individuos que sobreviviran y un porcentaje de elitismo. Seleccionaremos las mejores en base a nuestro porcentaje de elitismo, y completaremos la población que sobrevive con individuos alteatorios

In [31]:
def selection(solutions, surviving_number, elite_percent, cost_function=cost_function):
  # Se ordenan las soluciones en base a la funcion de coste
  solutions.sort(key=lambda x:cost_function(x))
  # Obtenemos nuestra población de elite
  elite_count = int(surviving_number * (elite_percent / 100))
  elite = solutions[:elite_count]
  not_elite = solutions[elite_count:]

  #Devolvemos la elite mas una selección alteatoria de los demas miembros
  return elite  + random.sample(not_elite, surviving_number - elite_count) 

In [32]:
# Prueba de nuestra selección
test_solutions = generate_random_solutions(5, 30, 6)
selected_solutions = selection(test_solutions, 2, 80)
for sol in selected_solutions:
  print(sol)

[{1, 5, 14, 15, 27}, {0, 3, 17, 25, 29}, {7, 10, 16, 18, 26}, {2, 4, 8, 11, 19}, {6, 9, 12, 20, 22}, {13, 21, 23, 24, 28}]
[{16, 3, 5}, {10, 19, 15}, {11, 20, 29}, {1, 6, 22}, {2, 13, 14}, {8, 12, 23}, {4, 21, 7}, {25, 27, 28}, {24, 0, 17}, {9, 26, 18}]


#### Cruce
Contaremos con un metodo de cruce, que se encargara de realizar los cruces de nuestra población y de un metodo de descendencia, que se encargara de crear una descendencia para dos soluciones. 

Para esta descendencia iremos combinando dia por dia respetando el orden. Ya que estamos haciendo uso de sets para representar las tomas a realizarse en el día, podemos estar seguros que esta combinación no nos devolvera repetidos en un mismo día. En caso de que nuestro dia termine con mas tomas de las necesarias, lo barajareemos y dividiremos en dos diferentes dias.

Al finalizar esta combinación, verificaremos que no estemos tomando la misma toma en mas de un día, en caso de ser así, la removeremos de la segunda ocurrencia.


Por otro lado nuestra función de cruce recibira una población, y devolvera la población ampliada con sus hijos.

In [33]:
def offspring(solution1, solution2, max_takes_on_day):
  offspring_solution = []
  ref_idx = min(len(solution1), len(solution2))
  for idx in range(ref_idx):
    new_day = solution1[idx] | solution2[idx]
    if len(new_day) > max_takes_on_day:
      # dividimos el set en dos diferentes dias
      takes_on_day = list(new_day)
      random.shuffle(takes_on_day)
      cut_index = len(takes_on_day) // 2
      offspring_solution.append(set(takes_on_day[cut_index:]))
      offspring_solution.append(set(takes_on_day[:cut_index]))
    else:
      offspring_solution.append(new_day)
  # Si existe un set que tenga mas elementos que ref_idx, entonces los agregamos indistintamente
  if len(solution1) > ref_idx:
    offspring_solution += solution1[ref_idx:]
  if len(solution2) > ref_idx:
    offspring_solution += solution2[ref_idx:]

  # Removemos las ocurrencias repetidas
  visited = set()
  for idx, day in enumerate(offspring_solution):
    new_day = day.copy()
    for take in day:
      if take in visited: new_day.remove(take)
      else: visited.add(take)
    offspring_solution[idx] = new_day
  # Quitamos los sets vacios
  return [x for x in offspring_solution if x]

In [34]:
# Prueba de nuestro metodo de descendencia
print(offspring([{1,2},{3,4},{5,6}],[{3,5},{1,4},{2,6}], 6))
print(offspring([{1,2},{3,4},{5,6}],[{3,5},{1,4},{2,6},{7,8},{9,10}], 6))
print(offspring([{1,2},{3,4},{5,6}],[{3,5},{1,4},{2,6},{7,8},{9,10}], 2))

[{1, 2, 3, 5}, {4}, {6}]
[{1, 2, 3, 5}, {4}, {6}, {8, 7}, {9, 10}]
[{1, 5}, {2, 3}, {4}, {6}, {8, 7}, {9, 10}]


In [36]:
def cross_over(solutions, max_takes_on_day):
  parents = copy.deepcopy(solutions)
  new_solutions = solutions

  while(len(parents) > 1):
    # seleccionamos 2 padres al azar
    p1, p2 = random.sample(parents, 2)
    new_solutions.append(offspring(p1, p2, max_takes_on_day))
    parents.remove(p1)
    parents.remove(p2)
  
  return new_solutions

In [38]:
# Prueba de nuestro metodo de cruce
after_cross_over = cross_over(
    generate_random_solutions(4,30 , 6),
    6
)

for sol in after_cross_over:
  print(sol)

[{0, 24, 7}, {10, 11, 13}, {16, 17, 29}, {1, 18, 2}, {3, 4, 12}, {25, 28, 20}, {5, 21, 22}, {26, 27, 23}, {8, 19, 15}, {9, 14, 6}]
[{16, 0, 18}, {9, 13, 1}, {27, 3, 5}, {28, 14, 15}, {23, 21, 7}, {8, 19, 6}, {24, 2, 10}, {11, 29, 22}, {25, 12, 17}, {26, 4, 20}]
[{2, 11, 13, 20, 26, 28}, {5, 7, 10, 19, 23, 29}, {8, 9, 16, 17, 18, 27}, {0, 1, 14, 15, 24, 25}, {3, 4, 6, 12, 21, 22}]
[{10, 11, 14, 16, 18, 21}, {0, 12, 17, 19, 20, 26}, {1, 2, 3, 7, 13, 15}, {6, 23, 24, 25, 27, 29}, {4, 5, 8, 9, 22, 28}]
[{16, 20, 26, 28, 13}, {0, 11, 18, 2}, {1, 29, 9, 10}, {19, 7, 5, 23}, {8, 27}, {17, 3}, {25}, {24, 14, 15}, {12, 6}, {4, 21, 22}]
[{0, 16, 18, 24, 14}, {10, 11, 21, 7}, {17, 20, 13}, {19, 26, 12}, {2, 3, 29, 15}, {1}, {23, 25}, {27, 6}, {8, 9, 5, 22}, {28, 4}]


#### Algoritmo

Haciendo uso de los metodos previamente implementados, implementaremos nuestro algoritmo genetico. Este generará una población inicial de un determinado tamaño, y durante n generaciones, la cruzara, evolucionara y seleccionara los mejores inidividuos. Iremos almacenando el mejor individuo encontrado en todo el proceso.

In [40]:
def genetic_algo(
    actors_by_take, # Input
    elite_percent, # Hiperparametro - Porcentaje de elitismo
    number_of_generations,  # Hiperparametro - Numero de generaciones a evalyar
    solutions_size, # Hipterparametro - Tamaño de la población

    number_of_takes = 30, # Numero de tomas, depende del problema
    max_takes_on_day = 6,  # Maximo numero de tomas por dia, depende del problema
    cost_function = cost_function, # Función de coste
    cross_over = cross_over, # Función de cruce
    selection = selection, # Función de selección
    generate_random_solutions = generate_random_solutions, # Función que genera soluciones aleatorias
    mutation = mutation, # Función que muta los elementos de nuestra población
  ): 

  solutions = generate_random_solutions(solutions_size, number_of_takes, max_takes_on_day)
  
  # obtenemos la mejor solucion de nuestra población
  best_sol = min(solutions, key = lambda x:cost_function(x))
  best_cost = cost_function(best_sol, actors_by_take)

  # iteramos las generaciones
  for generation in range(1,number_of_generations + 1):

    # Cruzamos nuestra población
    solutions = cross_over(solutions, max_takes_on_day)

    # Evolucionamos nuestra poblacion
    solutions = [mutation(x, max_takes_on_day) for x in solutions]

    # Seleccionamos los mejores individuos
    solutions = selection(solutions, solutions_size, elite_percent, cost_function=lambda x:cost_function(x, actors_by_take))

    # obtenemos la mejor solucion de nuestra población
    best_local_sol = min(solutions, key = lambda x:cost_function(x, actors_by_take))
    best_local_cost = cost_function(best_local_sol, actors_by_take)

    # si mejoramos la solucion, almacenamos
    if best_local_cost < best_cost:
      best_cost = best_local_cost
      best_sol = best_local_sol

  return best_sol, best_cost

In [41]:
# Tuneamos los hiperparametros para ver como nos va con diferentes pruebas. Lo volvemos una función para usarlo despues

def print_tunning_test(actors_by_take, generations, elite_percents, solutions_size):
  results = []

  for g in generations:
    for e in elite_percents:
      for s in solutions_size:
        sol = genetic_algo(actors_by_take, e, g, s)
        results.append({
            'Generaciones': g,
            'Elitismo': e,
            'Tamaño población': s,
            'Coste': sol[1],
            'Solución': sol[0],
        })

  # Reordenamos las soluciones de mejor a peor
  results.sort(key=lambda x:x['Coste'])

  for result in results:
    for key in result.keys():
      print(f"{key}: {result[key]}")
    print()

In [ ]:
generations = [100, 200]
elite_percents = [60, 90]
solution_sizes = [100, 2000]

print_tunning_test(actors_by_take, generations, elite_percents, solutions_size)


Generaciones: 200
Elitismo: 90
Tamaño población: 2000
Coste: 31
Solución: [{5, 7, 9, 11, 18, 25}, {0, 3, 10, 12, 16, 29}, {1, 4, 19, 21, 26, 27}, {2, 6, 8, 14, 15, 24}, {13, 17, 20, 22, 23, 28}]

Generaciones: 100
Elitismo: 60
Tamaño población: 2000
Coste: 32
Solución: [{4, 20, 27, 11, 28, 13}, {2, 3, 9, 10, 14, 25}, {5, 8, 12, 19, 24, 26}, {7, 17, 18, 21, 23, 29}, {0, 1, 6, 15, 16, 22}]

Generaciones: 100
Elitismo: 90
Tamaño población: 100
Coste: 32
Solución: [{17, 18, 19, 23, 28, 15}, {2, 3, 7, 9, 10, 25}, {0, 1, 12, 21, 22, 26}, {4, 14, 20, 24, 27, 29}, {5, 6, 8, 11, 13, 16}]

Generaciones: 100
Elitismo: 90
Tamaño población: 2000
Coste: 32
Solución: [{2, 19, 25, 9, 11, 14}, {7, 10, 13, 16, 20, 22}, {0, 1, 6, 23, 26, 28}, {3, 4, 8, 15, 24, 27}, {5, 12, 17, 18, 21, 29}]

Generaciones: 200
Elitismo: 60
Tamaño población: 2000
Coste: 32
Solución: [{17, 3, 20, 22, 23, 14}, {0, 1, 12, 16, 18, 21}, {2, 9, 15, 19, 24, 25}, {4, 6, 8, 10, 11, 13}, {5, 7, 26, 27, 28, 29}]

Generaciones: 100
Eli

### Calcula la complejidad del algoritmo

La complejidad de nuestro algoritmo para nuestro caso de estudio, es decir 30 tomas y 10 actores dependera del número de generaciones y del tamaño de la población. 

Nuestra iteración principal es el número de generaciones. Por cada generación se realizara la mutación, selección y cruce. En cada uno de estos procesos realizamos una iteración en toda la población y ya trabajamos sobre los valores. Las operaciones dadas dentro de nuestros auxiliares geneticos son indiferentes para nuestra dimensionalidad, es decir que podemos considerarlas como constantes multiplicativas.

Al final terminaremos con la siguiente complejidad:

```
O(g,p) = g*p
```
Donde g es el número de generaciones y p el tamaño de la población

Si consideramos el número de las tomas, es decir la dimensionalidad del problema, este tambien afectara a nuestro algoritmo y no podremos considerar las operaciones dadas en nuestros auxiliares geneticos como constantes multiplicativos. Vemos que la operación mas pesada que realizamos dentro de nuestros auxiliares es un sort, en la mutación. Este sort ordena los días en base al coste diario, en el peor de los casos podemos tener tantos dias como tomas. En los demás auxiliares,  solo se itera sobre el número de días. Es por esto que tomamos la complejidad del sort como la mas alta.


```
O(g,p) = g*p*nlogn
```
Donde g es el número de generaciones, p el tamaño de la población y n es el número de tomas.

El número de actores no afectara a nuestra complejidad temporal debido al uso de bits en la función de coste.



### Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Crearemos un metodo que pueda generar entradas aleatorias para cualquier numero de tomas/actores.

In [ ]:
def generate_input(number_of_takes, number_of_actors):
  # Necesitamos generar un array de enteros (Ver representación de datos)
  # Nuestro numero sera aleatorio y debe ser entre 1 (se necesita al menos un actor por toma) y el máximo número que se ppuede representar
  # con #actores bits (todos los actores son parte de la toma)
  bits = ''.join(['1' for _x in range(0, number_of_actors)])
  all_actors = int(bits, 2)

  #### Solo para mostrar la representación de manéra menos abstracta:
  input_arr = []
  print("Actores:  12345678910")
  print("Tomas")
  for take_number in range(0, number_of_takes):
    representation = random.randint(1,all_actors)
    input_arr.append(representation)

    string_rep = bin(representation).replace('0b','')
    # agregamos 0's al comienzo para la representación (si es necesario)
    string_rep += ''.join(['0' for _x in range(0, number_of_actors - len(string_rep))])
    print(f"Toma #{'0' if take_number + 1 < 10 else ''}{take_number + 1}: {string_rep}")

  print()
  return input_arr

In [ ]:
# Generamos el input para 30 tomas y 10 actores
input = generate_input(30, 10)
print(input)

Actores:  12345678910
Tomas
Toma #01: 1001100000
Toma #02: 1001110110
Toma #03: 1010111010
Toma #04: 1111101000
Toma #05: 1111001101
Toma #06: 1101010000
Toma #07: 1111101100
Toma #08: 1011001000
Toma #09: 1110111010
Toma #10: 1000000000
Toma #11: 1011010010
Toma #12: 1100010010
Toma #13: 1101101001
Toma #14: 1111100000
Toma #15: 1111001111
Toma #16: 1100001001
Toma #17: 1011010101
Toma #18: 1010111110
Toma #19: 1111001100
Toma #20: 1101111100
Toma #21: 1100011110
Toma #22: 1101001011
Toma #23: 1101100000
Toma #24: 1100001010
Toma #25: 1110001100
Toma #26: 1000000000
Toma #27: 1111001100
Toma #28: 1100111101
Toma #29: 1100110000
Toma #30: 1000011110

[38, 315, 698, 125, 973, 212, 502, 356, 954, 256, 361, 393, 873, 124, 975, 777, 725, 351, 972, 223, 798, 843, 216, 778, 227, 32, 972, 829, 51, 271]


### Aplica el algoritmo al juego de datos generado

In [ ]:
generations = [100, 200]
elite_percents = [60, 90]
solutions_size = [100, 2000]

print_tunning_test(input, generations, elite_percents, solutions_size)

Generaciones: 200
Elitismo: 90
Tamaño población: 2000
Coste: 44
Solución: [{20, 6, 8, 25, 27, 15}, {2, 5, 19, 22, 24, 28}, {4, 11, 14, 21, 23, 29}, {0, 1, 3, 7, 9, 17}, {10, 12, 13, 16, 18, 26}]

Generaciones: 100
Elitismo: 60
Tamaño población: 2000
Coste: 45
Solución: [{4, 9, 11, 15, 18, 26}, {1, 2, 3, 5, 6, 7}, {8, 12, 14, 16, 21, 22}, {13, 17, 19, 24, 28, 29}, {0, 10, 20, 23, 25, 27}]

Generaciones: 100
Elitismo: 90
Tamaño población: 2000
Coste: 45
Solución: [{0, 16, 2, 17, 4, 20}, {5, 6, 7, 19, 25, 28}, {11, 14, 18, 23, 26, 29}, {1, 3, 8, 21, 22, 24}, {9, 10, 12, 13, 15, 27}]

Generaciones: 200
Elitismo: 60
Tamaño población: 2000
Coste: 45
Solución: [{6, 14, 18, 19, 20, 29}, {1, 15, 23, 25, 27, 28}, {4, 7, 9, 10, 11, 26}, {0, 2, 5, 8, 13, 22}, {3, 12, 16, 17, 21, 24}]

Generaciones: 100
Elitismo: 60
Tamaño población: 100
Coste: 46
Solución: [{1, 10, 13, 15, 17, 27}, {0, 3, 5, 7, 24, 29}, {2, 6, 8, 12, 22, 28}, {9, 11, 14, 18, 21, 25}, {4, 16, 19, 20, 23, 26}]

Generaciones: 100
Eli

## Como es posible avanzar en el estudio del problema?

A continuación se presentan recomendaciones para avanzar con el estudio del problema:
<li><strong>Diferentes tamaños del problema: </strong></li> La implementación de nuestros algoritmos se penso para poder ser aplicado a diferentes tamaños del problema. Es decir el número de tomas y el número de actores puede ser modificado para su aplicación. En este estudio solo se evaluo el algoritmo genetico en la dimensionalidad inicial (30 tomas y 10 actores). Se recomienda seguir aplicando el algoritmo en diferentes dimensionalidades para ver como se comporta.
<li><strong>Mutación, Selección y Cruce: </strong></li>
Se probarón heuristicas especificas para Mutación, Selección y Cruce. Para evaluar si es posible obtener un mejor desempeño, se puede hacer uso de nuestro algoritmo genetico, el cual acepta como parametros otras funciones para estos procesos.
Para la función de <strong>Mutación</strong> estamos realizando agrupaciones al máximo numero de tomas por día. Podemos intentar diferentes agrupaciones, pese a que sabemos que la solución convergera en un maximo de tomas por cada día, el quitar esta heuristica puede ayudarmos a conocer otros espacios de solución. Para la <strong>selección</strong> estamos eligiendo la población de elite, y aumentando los demás miembros de manera aleatoria. Puede ser interesante la opción de probar un juego de ruleta, es decir, asignar probabilidad de selección a los demas individuos que no sean elite en base a su calidad como solución. Y por ultimo para la función de <strong>cruce</strong> se implemento tratando de combinar ambas soluciones y obtener un hijo. Se podría intentar obtener mas de un hijo a partir de estas combinaciones para ver si diversificamos nuestro espacio de solución.
<li><strong>Tuneo de hiperparametros: </strong></li>
El algoritmo genetico es capaz de recibir el porcentaje de elitismo, el número de generaciones y el tamaño de la población como hiperparametros. Se hicieron unas cuantas pruebas con estos hiperparametros para demostrar el algoritmo y la opción de ser tuneado. Se puede jugar con estos parametros para ver si encontramos mejores soluciones.
<li><strong>Set de soluciones iniciales: </strong></li>
Como se presento previamente, es posible asumir que nuestra solución optima toma 6 tomas por día, cada día. Podemos ver de iniciar solo con soluciones que tomen en cuenta este factor para ver si mejora neustros resultados. Se tomarón soluciones iniciales de diferentes tamaños en esta implementación ya que creo que puede ayudarnos en el proceso de diversificación, al menos con nuestra implementación actual de mutación y cruce, pero sería bueno probar esta hipotesis empiricamente.